In [ ]:
#hide
# Have to do this to avoid the performance degradation reported in https://forums.fast.ai/t/performance-degradation-between-fastai-2-2-5-and-2-2-7/86069
!pip install fastai==2.2.5

In [2]:
from fastai.vision.all import *

In [3]:
path = untar_data(URLs.PASCAL_2007)

In [4]:
def get_x(r): return (path/"train") / r["fname"]
def get_y(r): return r["labels"].split(" ")
def splitter(df):
  train = df.index[~df["is_valid"]].tolist()
  valid = df.index[df["is_valid"]].tolist()
  return train,valid
  
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock), 
                         get_x=get_x, 
                         get_y=get_y, 
                         splitter=splitter,
                         item_tfms= RandomResizedCrop(128, min_scale=0.35))

In [5]:
df = pd.read_csv(path / "train.csv")
dls = dblock.dataloaders(df)

In [6]:
dls.show_batch()

In [8]:
dls.one_batch()[1].type()

'torch.cuda.FloatTensor'

## CNN Learner

This is pretty much exactly what is done in the book:

In [10]:
learn = cnn_learner(dls, resnet18, metrics=[accuracy_multi])
learn.fine_tune(3, freeze_epochs=2)

95% accuracy was great for the other datasets, but it doesn't really say much about how well we do in this problem. Even a model that just predicts 0 for every label (see below) gets 92% accuracy.

## Naive baseline

In [ ]:
class AllZeroModel(nn.Module):
    def __init__(self):
        super(AllZeroModel, self).__init__()

    def forward(self, x):
        output = torch.zeros(x.shape[0], 20).to("cuda:0")
        return output

In [ ]:
model = AllZeroModel()

In [ ]:
accuracy_no_sigmoid = partial(accuracy_multi, sigmoid=False)
dumb_learner = Learner(dls, AllZeroModel(), metrics=[accuracy_no_sigmoid])

In [ ]:
_, multi_accuracy = dumb_learner.validate()

In [ ]:
print(f"Multi Accuracy: {multi_accuracy}")

Multi Accuracy: 0.922390341758728


This shows that accuracy_multi is not an appropriate performance metric for this task. 

In [ ]:
dumb_learner.show_results()

## Attempt to use APScoreMulti

In [17]:
learn = cnn_learner(dls, resnet34, metrics=[accuracy_multi, APScoreMulti()])
learn.fine_tune(15, freeze_epochs=5)

Ahh, this makes more sense, SOTA is ~95% mAP ([source](https://arxiv.org/pdf/2009.14119.pdf)). Okay, I'm happy with this.

## GUI to interact with model

In [ ]:
from ipywidgets import FileUpload, Output, Label, VBox, Button

In [ ]:
upload = FileUpload()
out_pl = Output()
lbl_pred = Label()
btn_run = Button(description="classify")

def on_click_classify(change):
  img = PILImage.create(upload.data[-1])
  out_pl.clear_output()
  with out_pl: display(img.to_thumb(128, 128))
  pred, pred_idx, probs = learn.predict(tensor(img))
  lbl_pred.value = f'Prediction {pred}, Probability: {probs[pred_idx]}'

btn_run.on_click(on_click_classify)

In [ ]:
VBox([upload, btn_run, out_pl, lbl_pred])

In [ ]:
learn.predict(upload.data[-1])
print(pred)
print(list(zip(dls.vocab, probs)))